# 红移和模板拟合

###CELL###

本笔记本涵盖了一些基本示例，展示用户如何使用可视化工具 [Jdaviz](https://jdaviz.readthedocs.io/en/latest/) 或通过编程方式使用 [Specutils](https://specutils.readthedocs.io/en/latest/) 来测量源的红移。

**使用案例：** 使用两种不同的方法从其光谱中测量一个星系的红移。

**数据：** JWST/NIRSpec 来自项目 2736 的光谱。

**工具：** jdaviz, specutils。

**跨仪器：** NIRISS, NIRCam。

**内容**

- [资源和文档](#resources)

- [安装](#installation)

- [导入](#imports)

- [获取示例数据](#data)

- [用 Specviz “肉眼” 测量红移](#byeye)

- [使用交叉相关方法测量红移](#crosscorr)

    - [获取模板并为使用做准备](#get_template)

    - [从观测光谱中减去连续谱](#continuum)

    - [清理光谱](#cleanup)

    - [运行交叉相关函数](#run_crosscorr)

**作者**：Camilla Pacifici (cpacifici@stsci.edu)<br>

**更新日期**：2024年11月18日

<a id='resources'></a>

## 资源和文档

本笔记本使用了来自 [Specutils](https://specutils.readthedocs.io/en/stable/) 和 [Jdaviz](https://jdaviz.readthedocs.io/en/latest/) 的功能。空间望远镜科学研究所的开发人员可通过 [JWST Help Desk](https://jwsthelp.stsci.edu/) 解答问题并解决问题。如果您希望发送反馈或报告问题，您也可以直接在 Github 上提交问题，适用于 [Specutils](https://github.com/astropy/specutils) 和 [Jdaviz](https://github.com/spacetelescope/jdaviz)。

###CELL###

<a id='installation'></a>

## 安装

本笔记本摘自 [JWebbinar 材料](https://www.stsci.edu/jwst/science-execution/jwebbinars)。

要运行此笔记本，您需要创建一个包含 jdaviz 包的环境，按照以下指令操作。

`conda create -n jdaviz python=3.11`  

`conda activate jdaviz`  

从最新版本安装  

`pip install jdaviz`  

或者通过 git 安装  

`pip install git+https://github.com/spacetelescope/jdaviz.git`

###CELL###

<a id='imports'></a>

## 导入

###CELL###

In [ ]:
# 导入操作系统相关的临时文件处理库
import tempfile

# 导入numpy库，用于数值计算
import numpy as np

# 导入astroquery库中的Observations模块，用于获取天文观测数据
from astroquery.mast import Observations

# 导入jdaviz库，用于后续获取版本号
import jdaviz
# 导入Specviz模块，用于天文光谱可视化
from jdaviz import Specviz

# 导入astropy库，用于后续获取版本号
import astropy
# 导入astropy库的单位模块
import astropy.units as u
# 导入astropy库的ascii和fits模块，用于数据读取和文件操作
from astropy.io import ascii, fits
# 导入astropy库的数据下载模块
from astropy.utils.data import download_file
# 导入astropy库的模型模块，用于创建线性模型
from astropy.modeling.models import Linear1D
# 导入astropy库的标准偏差不确定性模块
from astropy.nddata import StdDevUncertainty

# 导入specutils库，用于后续获取版本号
import specutils
# 导入specutils库的Spectrum1D和SpectralRegion模块，用于光谱数据处理
from specutils import Spectrum1D, SpectralRegion
# 导入specutils库的拟合模块，用于光谱连续谱拟合
from specutils.fitting import fit_generic_continuum
# 导入specutils库的分析模块，用于光谱数据相关性分析
from specutils.analysis import correlation
# 导入specutils库的操作模块，用于光谱区域提取
from specutils.manipulation import extract_region

# 导入glue库的roi模块，用于定义光谱范围
from glue.core.roi import XRangeROI

# 导入matplotlib库的pyplot模块，用于绘图
from matplotlib import pyplot as plt

# 导入IPython的显示模块，用于在Jupyter环境中显示HTML内容
from IPython.display import display, HTML

In [ ]:
# 自定义matplotlib样式

# 设置默认的图形大小为10x5英寸
plt.rcParams["figure.figsize"] = (10, 5)

# 定义一组新的参数来更新matplotlib的配置
params = {'legend.fontsize': '18',  # 图例字体大小
          'axes.labelsize': '18',  # 轴标签字体大小
          'axes.titlesize': '18',  # 轴标题字体大小
          'xtick.labelsize': '18',  # x轴刻度标签字体大小
          'ytick.labelsize': '18',  # y轴刻度标签字体大小
          'lines.linewidth': 2,     # 线条宽度
          'axes.linewidth': 2,      # 轴线宽度
          'animation.html': 'html5',  # 动画格式
          'figure.figsize': (8, 6)}  # 图形大小为8x6英寸

# 使用上面定义的参数更新matplotlib的配置
plt.rcParams.update(params)

# 更新配置，设置最大打开警告为0，避免打开过多图形时的警告
plt.rcParams.update({'figure.max_open_warning': 0})

# 这确保了我们的notebook使用浏览器的全部宽度
display(HTML("<style>.container { width:100% !important; }</style>"))

###CELL###

### 版本：

###CELL###

In [ ]:
# 打印 jdaviz 库的版本号
print("jdaviz:", jdaviz.__version__)

# 打印 astropy 库的版本号
print("astropy:", astropy.__version__)

# 打印 specutils 库的版本号
print("specutils:", specutils.__version__)

###CELL###

<a id='data'></a>

## 获取示例数据

在此我们从[早期发布观测数据](https://www.stsci.edu/jwst/science-execution/approved-programs/webb-first-image-observations)计划2736下载一个光谱，以及我们将用作红移测量模板的模型光谱。该模板基于[Pacifici et al. (2012)](https://ui.adsabs.harvard.edu/abs/2012MNRAS.421.2002P/abstract)中完成的包含发射线的简单恒星族群模型的组合。

###CELL###

In [ ]:
# 选择机器上的特定目录或临时目录

data_dir = tempfile.gettempdir()  # 使用系统的临时目录

# 从MAST获取文件

fn = "jw02736-o007_s000009239_nirspec_f170lp-g235m_x1d.fits"  # 定义文件名

# 下载文件并保存到指定的本地路径
result = Observations.download_file(f"mast:JWST/product/{fn}", local_path=f'{data_dir}/{fn}')

# 下载模板文件
fn_template = download_file('https://stsci.box.com/shared/static/3rkurzwl0l79j70ddemxafhpln7ljle7.dat', cache=True)  # 下载并缓存文件

Jdaviz 默认读取表面亮度扩展（单位为 MJy/sr），但我更喜欢使用流量扩展（单位为 Jy）。我自己读取文件，创建了一个 Spectrum1D 对象。

###CELL###

In [ ]:
# 使用astropy库打开FITS文件
hdu = fits.open(f'{data_dir}/{fn}')

# 从FITS文件的第二个HDU中读取波长数据，并赋予相应的单位
wave = hdu[1].data['WAVELENGTH'] * u.Unit(hdu[1].header['TUNIT1'])

# 从FITS文件的第二个HDU中读取流量数据，并赋予相应的单位
flux = hdu[1].data['FLUX'] * u.Unit(hdu[1].header['TUNIT2'])

# 由于FLUX_ERROR数据全为nan，这里定义一个人工的不确定性
# std = hdu[1].data['FLUX_ERROR'] * u.Unit(hdu[1].header['TUNIT3']) # 这些都是nan，定义一个人工的不确定性

# 创建一个Spectrum1D对象，包含波长、流量和不确定性
fluxobs = Spectrum1D(spectral_axis=wave,
                     flux=flux,
                     uncertainty=StdDevUncertainty(0.1*flux))  # 使用流量的10%作为标准偏差的不确定性

# 输出Spectrum1D对象
fluxobs

<a id='byeye'></a>

## 通过肉眼测量红移与 Specviz

Specviz 将允许您将线波长与您在光谱中看到的发射线匹配。您可以使用 Line List 插件中的[红移滑块](https://jdaviz.readthedocs.io/en/latest/specviz/plugins.html#redshift-slider)来完成这一操作。但首先，让我们[在 Specviz 中打开光谱](https://jdaviz.readthedocs.io/en/latest/specviz/import_data.html)。

###CELL###

In [ ]:
# Call the app

viz = Specviz()  # 创建 Specviz 类的实例

viz.show()  # 显示实例的视图

###CELL###

In [ ]:
# 加载光谱数据

# viz.load_data(f'{data_dir}/{fn}', data_label="NIRSpec") # 从文件直接加载表面亮度数据

viz.load_data(fluxobs, data_label='NIRSpec')  # 从变量fluxobs加载数据，数据标签为'NIRSpec'

###CELL###

现在我们需要：

- 打开["线列表"插件](https://jdaviz.readthedocs.io/en/latest/specviz/plugins.html?highlight=plugin#line-lists)

<img src='./line_list.png' alt="线列表插件位于右侧菜单中" width="500"/>

- 选择预加载的线或添加自定义线（这些线不会在查看器中显示，因为它们是在静止帧中绘制的）

    - 提示：选择Ha和周围的NII线

<img src='./select_list.png' alt="选择SDSS列表" width="500"/>

<img src='./select_three_lines.png' alt="选择两条[NII]线和H alpha线" width="500"/>

- 输入一个猜测的红移值

<img src='./guess_redshift.png' alt="输入红移的猜测值（redshift=2.4）" width="500"/>

- 移动滑块以获得更好的匹配

<img src='./adjust_redshift_slider.png' alt="调整红移滑块以匹配光谱中的发射线" width="500"/>

- 使用[缩放工具](https://jdaviz.readthedocs.io/en/latest/specviz/displaying.html#pan-zoom)获得更精确的匹配

<img src='./precise_adjust_with_zoom.png' alt="使用缩放工具找到与发射线更好的匹配" width="500"/>

<a id='crosscorr'></a>

## 使用交叉相关法测量红移

在天文学中，使用交叉相关算法来测量红移是非常常见的。IRAF在其[xcsao](http://tdc-www.harvard.edu/iraf/rvsao/xcsao/xcsao.html)任务中使用了这种方法。在这里，我们使用Specutils的[模板交叉相关](https://specutils.readthedocs.io/en/stable/analysis.html#template-cross-correlation)功能来推导我们源的红移。在运行相关算法之前，我们需要做几件事：

- 获取用于相关的模板光谱

- 从模板和观测光谱中减去连续谱

- 确保光谱在波长上有一些重叠

###CELL###

<a id='get_template'></a>

### 获取模板并准备使用

模板用于交叉相关，因此可以为了方便而重新标准化。单位必须与观测光谱的单位匹配。我们可以在 erg/(s cm^2 A) 中进行连续谱减法，因为连续谱接近线性，然后通过 Jdaviz 运行以获得适当的转换。

###CELL###

In [ ]:
# 定义单位
spec_unit = u.erg / u.s / u.cm**2 / u.AA

# 使用 ascii 函数读取光谱数据
template = ascii.read(fn_template)

# 创建 Spectrum1D 对象
template = Spectrum1D(spectral_axis=template['col1']/1E4*u.um,  # 将第一列数据转换为微米单位
                      flux=(template['col2']/1E24)*spec_unit)  # 将第二列数据进行归一化，以匹配观测光谱的单位

In [ ]:
# Cut to useful range - template and obs MUST overlap, so we go to 2.4um.

# 定义一个布尔数组，用于筛选波长范围在0.45到2.4微米之间的模板数据
use_tmp = (template.spectral_axis.value > 0.45) & (template.spectral_axis.value < 2.4)

# 使用筛选后的布尔数组来裁剪模板的光谱轴和通量数据
template_cut = Spectrum1D(spectral_axis=template.spectral_axis[use_tmp], flux=template.flux[use_tmp])

###CELL###

In [ ]:
# 导入matplotlib.pyplot模块用于绘图
import matplotlib.pyplot as plt

# 创建一个图形窗口，并设置大小为10x6英寸
plt.figure(figsize=[10, 6])

# 绘制光谱数据，横坐标为波长，纵坐标为流量
plt.plot(template_cut.spectral_axis, template_cut.flux)

# 设置x轴标签，显示波长单位
plt.xlabel("wavelength ({:latex})".format(template_cut.spectral_axis.unit))

# 设置y轴标签，显示流量单位
plt.ylabel("flux ({:latex})".format(template_cut.flux.unit))

# 设置图形的标题为"Template"
plt.title("Template")

# 显示图形
plt.show()

此图显示了模板光谱（流量与波长关系），波长范围扩展到2.4微米，以便与观测光谱有一些波长重叠。

###CELL###

In [ ]:
# 导入连续谱减法所需的库
from astropy.modeling.models import Linear1D
from specutils import Spectrum1D, fit_generic_continuum
import matplotlib.pyplot as plt

# 创建一个掩码以选择特定波长范围的数据
mask_temp = ((template_cut.spectral_axis.value > 0.70) & (template_cut.spectral_axis.value < 2.40))

# 使用掩码创建一个新的Spectrum1D对象，仅包含用于连续谱拟合的数据
template_forcont = Spectrum1D(spectral_axis=template_cut.spectral_axis[mask_temp], flux=template_cut.flux[mask_temp])

# 使用fit_generic_continuum函数拟合连续谱，这里使用线性模型
fit_temp = fit_generic_continuum(template_forcont, model=Linear1D())

# 使用拟合的连续谱模型计算整个波长范围的连续谱
cont_temp = fit_temp(template_cut.spectral_axis)

# 从原始光谱中减去拟合的连续谱，得到连续谱被减去后的光谱
template_sub = template_cut - cont_temp

# 创建一个图形窗口，设置大小为10x6英寸
plt.figure(figsize=[10, 6])

# 绘制原始光谱和连续谱
plt.plot(template_cut.spectral_axis, template_cut.flux, label='Original Spectrum')
plt.plot(template_cut.spectral_axis, cont_temp, label='Fitted Continuum')

# 设置x轴和y轴的标签，使用光谱的单位
plt.xlabel("wavelength ({:latex})".format(template_sub.spectral_axis.unit))
plt.ylabel("flux ({:latex})".format(template_sub.flux.unit))

# 设置图形的标题
plt.title("Plot template and continuum")

# 显示图例
plt.legend()

# 显示图形
plt.show()

### CELL ###
图表显示了模板光谱和最佳拟合连续谱。

In [ ]:
# 打印 Spectrum1D 对象

print(template_sub)  # 打印 Spectrum1D 类型的对象 template_sub

###CELL###

In [ ]:
# 导入matplotlib.pyplot模块用于绘图
import matplotlib.pyplot as plt

# 创建一个图形窗口，并设置大小为10x6英寸
plt.figure(figsize=[10, 6])

# 绘制光谱数据，横坐标为波长，纵坐标为流量
plt.plot(template_sub.spectral_axis, template_sub.flux)

# 设置x轴标签，显示波长单位
plt.xlabel("wavelength ({:latex})".format(template_sub.spectral_axis.unit))

# 设置y轴标签，显示流量单位
plt.ylabel("flux ({:latex})".format(template_sub.flux.unit))

# 设置图形标题为“Continuum subtracted template”
plt.title("Continuum subtracted template")

# 显示图形
plt.show()

此图展示了在减去拟合连续谱后的模板光谱（流量与波长）。

###CELL###

<a id='continuum'></a>

### 从观测光谱中减去连续谱

我们可以采用不同的方法，并使用 [SpectralRegion](https://specutils.readthedocs.io/en/stable/spectral_regions.html) 来实现。如果观测光谱中未包含不确定性，我们还需要加入这一部分。

###CELL###

In [ ]:
# 定义光谱区域
region = SpectralRegion(2.0*u.um, 3.0*u.um)  # 创建一个从2.0微米到3.0微米的光谱区域

# 提取区域
spec1d_cont = extract_region(fluxobs, region)  # 从fluxobs中提取定义的光谱区域

# 运行拟合函数
fit_obs = fit_generic_continuum(spec1d_cont, model=Linear1D(5))  # 使用线性模型拟合光谱连续体

# 应用到光谱轴
cont_obs = fit_obs(fluxobs.spectral_axis)  # 使用拟合结果计算fluxobs的光谱轴上的连续体值

# 减去连续体
spec1d_sub = fluxobs - cont_obs  # 从原始光谱数据中减去连续体，得到去连续体后的光谱数据

In [ ]:
# 导入matplotlib.pyplot库用于绘图
import matplotlib.pyplot as plt

# 创建一个图形窗口，设置大小为10x6英寸
plt.figure(figsize=[10, 6])

# 绘制一维光谱数据，横轴为波长，纵轴为流量
plt.plot(spec1d_sub.spectral_axis, spec1d_sub.flux)

# 设置x轴标签，显示波长单位
plt.xlabel("wavelength ({:latex})".format(spec1d_sub.spectral_axis.unit))

# 设置y轴标签，显示流量单位
plt.ylabel("flux ({:latex})".format(spec1d_sub.flux.unit))

# 设置图形标题
plt.title("Continuum subtracted observed spectrum")

# 显示图形
plt.show()

此图显示了减去拟合连续谱后观测到的光谱（通量与波长）。

###CELL###

<a id='cleanup'></a>

### 清理光谱

最好去除可能看起来像发射/吸收线的伪迹，并且去除大的间隙。可以使用图形用户界面选择光谱的干净部分。如果不手动执行此操作，以下单元格将以编程方式处理。

<img src='./select_region.png' alt="选择光谱的干净区域" width="500"/>

###CELL###

In [ ]:
viz2 = Specviz()  # 创建一个Specviz的实例，用于数据可视化

viz2.load_data(spec1d_sub, data_label='spectrum continuum subtracted')  # 加载数据，数据标签为'spectrum continuum subtracted'

viz2.show()  # 显示可视化界面

In [ ]:
# 尝试获取特定区域的子集，如果手动未创建则捕获异常
try:
    # 从viz2对象获取指定标签和子集的光谱数据
    region1 = viz2.get_spectra(data_label='spectrum continuum subtracted', subset_to_apply='Subset 1')
    # 打印获取的区域数据
    print(region1)
    # 设置标志变量，表示区域存在
    region1_exists = True
except Exception:
    # 如果出现异常，打印提示信息
    print("There are no subsets selected.")
    # 设置标志变量，表示区域不存在
    region1_exists = False

# 如果指定的光谱区域不存在，则进行遮罩处理
if region1_exists is False:
    # 获取名为'spectrum-viewer'的视图对象
    sv = viz2.app.get_viewer('spectrum-viewer')
    # 清空当前选中的子集
    sv.toolbar_active_subset.selected = []
    # 应用一个区域兴趣（ROI），用于遮罩2.24到3.13范围内的数据
    sv.apply_roi(XRangeROI(2.24, 3.13))

In [ ]:
# Get spectrum out with mask

# 从viz2对象中获取光谱区域
spec1d_region = viz2.get_spectral_regions()

# 使用指定的区域（'Subset 1'）从spec1d_sub中提取遮罩后的光谱
spec1d_masked = extract_region(spec1d_sub, spec1d_region['Subset 1'], return_single_spectrum=True)

###CELL###

我们在一个新的Specviz实例中可视化观测到的和模板的连续谱减去背景的光谱。点击“Home”按钮查看整个波长范围。模板光谱将改变单位以匹配观测光谱。

###CELL###

In [ ]:
# 创建Specviz对象的实例
viz3 = Specviz()

# 加载带有掩码的一维光谱数据，并给这份数据命名为'observation'
viz3.load_data(spec1d_masked, data_label='observation')

# 加载模板子集数据，并给这份数据命名为'template'
viz3.load_data(template_sub, data_label='template')

# 显示可视化界面
viz3.show()

In [ ]:
# 从viz3模块获取数据，使用显示单位
template_newunit = viz3.get_data('template', use_display_units=True)

# 输出获取的数据
template_newunit

###CELL###

<a id='run_crosscorr'></a>

### 运行交叉相关函数

###CELL###

In [ ]:
# 调用函数计算模板相关性
corr, lag = correlation.template_correlate(spec1d_masked, template_newunit, lag_units=u.one)

# 绘制相关性图
plt.plot(lag, corr)  # 绘制lag与corr的关系图

plt.xlabel("Redshift")  # 设置x轴标签为“Redshift”
plt.ylabel("Correlation")  # 设置y轴标签为“Correlation”

plt.show()  # 显示图像

此图显示了相关值与红移的关系。峰值（在红移2.5附近）表明观测到的光谱和模板光谱的相关性最佳的位置。

###CELL###

In [ ]:
import numpy as np  # 导入numpy库，用于数组和数学运算

# 假设corr和lag是已经定义好的数组
# corr是某种相关性数据的数组，lag是对应的滞后数组

index_peak = np.argmax(corr)  # 找到corr数组中最大值的索引

z = lag[index_peak]  # 使用找到的索引从lag数组中获取对应的值

print("Redshift from peak maximum: ", z)  # 打印计算得到的红移值

In [ ]:
# Redshifted template_sub

# 创建一个新的Spectrum1D对象，其光谱轴经过红移调整，流量保持不变
template_sub_z = Spectrum1D(spectral_axis=template_sub.spectral_axis * (1. + z),
                            flux=template_sub.flux)

###CELL###

In [ ]:
# 导入Specviz库用于数据可视化
from jdaviz import Specviz

# 创建Specviz的实例
viz4 = Specviz()

# 加载并标记观测到的光谱数据
viz4.load_data(spec1d_masked, data_label='Observed spectrum')

# 加载并标记经过红移处理的模板光谱数据
viz4.load_data(template_sub_z, data_label='Redshifted best template')

# 显示可视化界面
viz4.show()

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>

###CELL###